In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sklearn


In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras import layers, models

## COMP 3132 -  Machine Learning with Python
__Week 12 lecture__

### [github.com/kamrik/ML2](https://github.com/kamrik/ML2)

### [slido.com/COMP3132](http://slido.com/COMP3132)


## Plan for today
 - Home assignment, due the day before the final exam
 - Using embeddings
 - Dropout & regularization

## Home assignment

## N-grams
 * https://books.google.com/ngrams/

## Word embeddings

![](https://files.knime.com/sites/default/files/styles/inline_medium/public/word_embedding_-_word2vec_explained_-_figure1_0.png?itok=h_eMBl8O)

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/06/06062705/Word-Vectors.png)

## Recurrent Neural Nets - RNNs

![](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/05231650/rnn-neuron.png)

In [ ]:
# Given weather measurement from several days, predict the temperature a day later
model = Sequential()

model.add(layers.GRU(32, input_shape=(None, n_features)))

model.add(layers.Dense(1))

![](https://deeplearning4j.org/images/guide/rnn_data.png)